In [1]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    if "2.Final_output" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [3]:
import os
from Bio import SeqIO
import subprocess

family_group = "5.Coliadinae"

output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}"
make_output_folder(output_location)
genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_species = os.listdir(f"{output_location}/1.Blast_result")
if 'desktop.ini' in list_of_species:
    list_of_species.remove("desktop.ini")
# print(list_of_species)
# list_of_species = ["Heliconius_charithonia"]
# list_of_species = ["Maniola_hyperantus"]
error_exons = ''
for species in list_of_species:
    print(species)
    genome_file = get_genome_file(genome_location,species)
    print(genome_file)
    previous_coordinate_start = ''
    previous_scaffold = ''
    with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
        coordinate_file_list = coordinate_final_file.readlines()
    
    for i in range(1,len(coordinate_file_list)):
        
        coordinate_line_split = coordinate_file_list[i].split(",")
        
            
#         print(coodinate_line_split[5])
        if coordinate_line_split[5] == "N":
            if previous_coordinate_start == '':
                previous_coordinate_start = coordinate_line_split[2]
                previous_scaffold = coordinate_line_split[1]
            complement = coordinate_line_split[4]
            start,stop = coordinate_line_split[2],coordinate_line_split[3]
            print(start,stop)
            query_name = coordinate_line_split[6]
            scaffold = coordinate_line_split[1]
            exon = f"Exon_{query_name.split('_')[-1]}"
            left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
            print(left_oh, right_oh)
#             print(complement)
            if complement == "0":
                if start < previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            elif complement == "1":
                if start > previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            else:
                print("ERROR")
            
        elif coordinate_line_split[5] == "Y":
            query_name = coordinate_line_split[6]
            exon = f"Exon_{query_name.split('_')[-1]}"
            error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            continue
            
        gene_sequence = get_gene_sequence(genome_location, 
                                          species, 
                                          genome_file, 
                                          scaffold,
                                          start,
                                          stop,
                                          complement,
                                          output_location)
        
       
        current_exon = query_name.split('_')[-1]
        if current_exon == "1":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()
            print(translated)
            if "*" in translated:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
        
                write_exon_output(output_location,species,exon,exon_output)
        elif current_exon == "16":
            if complement == 1:
                translated = gene_sequence[:-int(left_oh)].reverse_complement.translate()
                print(gene_sequence)
            else:
                translated = gene_sequence[int(left_oh):].translate()
                print(gene_sequence)

            if "*" in translated[:-1]:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                print(translated)
            else:
                
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
                write_exon_output(output_location,species,exon,exon_output)
#         assert False
        else:
            exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
            print(exon_output)

            write_exon_output(output_location,species,exon,exon_output)
        
#         assert False
        previous_coordinate_start = start
        previous_scaffold = scaffold
    error_exons += "\n\n"
#     print(coordinate_file_list)
#     break
coordinate_file = ''
print(error_exons)
with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
    error_out_file.write(error_exons)

Colias_behrii
GCA_029959065.1_ilColBehr1.0.hap2_genomic.fna
6804607 6804703
0 1
Getting Gene
MEWVLRGPQIHSTFSSLGFQHAEGYHINENCN
>Colias_behrii_Exon_1_JARWMC010000003.1_6804607_6804703_left_0_right_1
ATGGAGTGGGTTCTTAGGGGACCTCAGATTCATAGTACATTCAGTAGTCTTGGATTTCAACATGCAGAAGGATATCACATAAATGAAAATTGCAATG
6805578 6806125
2 0
Getting Gene
>Colias_behrii_Exon_2_JARWMC010000003.1_6805578_6806125_left_2_right_0
CTGCGTTGGAGTCCATTCTTCATAACATTCTAACGGAGGATAAAAATTTACGCACTTATCGTAGAAGCATAAGTTTTGGACAAAACATCAAGAAAGATATCATACCCCTTTTAATAAATGTTAAAGATAATAAAACGATCGAGTTACTCATCAAGATACTTGTGAATTTAACTATACCTGTTGAATGTTTACTTTCCGTGGAAACGATATCACACACCGATATCGGAAGGCACACTATATTTGAAATCAACAGCCTTCTCGCTTCTACTAAATCTGCATTTATTGACCATCGGGCTACGAAAGTGGTCATAGAGTTTCTGAAGAAGAACTCGGATGTTGAACAGAAGCCGAAATTATCTTCAGAGAGTTGTACTAACATCAGTAACACTCTACTGTTCTTACGGAATATTCTTCATATCCCAGAAGACGTGAACAGTATTTCAAATTATAACAATCCTCCACATACTATTCAAAATCAAATATTATGGAATATTTTTAGTCAAAGCATTGACAAAATTCTCATTAAATTAATGACGATCCCAGATTCG
6806635 6806788
0 1
Getting Gene
>Colias_behrii_Exon

>Colias_croceus_Exon_8_NC_059539.1_6064225_6064325_left_0_right_2
GACCTCGTGGTTACGAATCACATTTTATTGATGTTTTTGGACAGTGTTATGAAATTACCAGATTACAAGGGATCTGCGAAATTAATTGACCACATCAAGCA
6064592 6064790
1 0
Getting Gene
>Colias_croceus_Exon_9_NC_059539.1_6064592_6064790_left_1_right_0
gtTCGCTACACCAGAGGTTATGTACCAGTATGGACTCTTGTTAGAAGATTACGAAGGCAACGGAGAATTTGTAAATGATTGCGTATTCACGTTGATGCACCATGTTGGCGGTGACTTGGGCTGTTTGATTTCACTCTTCCAAccgaaaatattgaaaactttTACATCTATATGGAAATCTGAGTTTGAAATTTGCGAT
6064946 6065136
0 2
Getting Gene
>Colias_croceus_Exon_10_NC_059539.1_6064946_6065136_left_0_right_2
gatTGGTCCGATCTAATAGAATatgttataaatacatttgtaaaGAAACCGCATACGTTACAACGTACGGCAAGCTTCCGATTGGATACAGAAAGTTTTGATGACGACAAAATTCTTAAAGAACttattgaaaatgaagatGTGTCCTCTAGTAGGGCCATTGTCAACTATGATGCAAGCGTAAAGGACAg
6065343 6065833
1 1
Getting Gene
>Colias_croceus_Exon_11_NC_059539.1_6065343_6065833_left_1_right_1
GTGGACAGACGACGAATTATCAGCATTTAATTGGATATACATGCAAAGTAGCACATCAACAGATACTGTGGGTGAAATTGTAAGATTATTTAAAGAAGATGGCATTATCAAAACTCGCGACGCGGTCATTAGGCAGCTCCA

GCA_907164665.1_Colias_nastes_genomic.fna
Eurema_hecabe
GCA_026213755.1_CUHK_EHE_v1_genomic.fna
5521427 5521523
0 1
Getting Gene
MEWVLRGPQIHSTFSDLGYAHPEGYHVNENCN
>Eurema_hecabe_Exon_1_JADANM010006787.1_5521427_5521523_left_0_right_1
ATGGAGTGGGTTCTTCGGGGACCACAGATACATAGCACATTCAGCGATCTTGGATATGCACATCCAGAAGGATATCACGTAAATGAGAACTGTAAtg
5520549 5521099
2 0
Getting Gene
>Eurema_hecabe_Exon_2_JADANM010006787.1_5520549_5521099_left_2_right_0
CGACCTTGGAGTcaattcttcataatatactaaatgagGACAAAAATTTACGAACGTATCGCAGGAGTATAAATTTCGGACAAAACatcaataaagatattattcctctattaataaatgtgaaagagGATAAAACAACAGAATTGTTGATTCAAATTCTTGTAAATCTGACAATACCTGTCGAGTGTTTGTTATCTGTTGAAACTATATCGAACACTGACATTGGAAGACATACTATATTCGAAATAAACGGCTTGTTGGGCTCTGCAAAATCCGCCTTTATAGATCATAGAGCAACGAAGGTTGTCAtcgagtttttaaaaaaaaattcagatGTTGAACAGACGGCTAAATTGTCTGCAGAGCAGTGCACAAATATCAGTAATACTCTTCTGTTTTTACGTAATATTCTTCATATACCGGAGGATGTGAATAGTATTTCAACAAACTCTAATGGACCTCCACatacaattcaaaacaaaatattatggaaTCTTTTTAGCCAAAGTATAGACAGAATACTTATAAAACTCATGACGATACCGGACTCG
5519252 55194

>Zerene_cesonia_Exon_8_NC_052102.1_6254628_6254728_left_0_right_2
gACCTTGTGGTTACGAATCACATTTTACTAATGTTTTTGGACAGTGTTATGAAATTACCGGATTACAAGGGATCTGGAAAATTAGTTGACCATATAAAGCA
6254219 6254417
1 0
Getting Gene
>Zerene_cesonia_Exon_9_NC_052102.1_6254219_6254417_left_1_right_0
gttCGCCACTCCAGAGATTATGTACCAATATGGTCTCCTATTAGAAGATTACGAAGGGAACGGAGAGTTCGTGAATGATTGCGTTTTCACGTTGATGCACCATGTCGGCGGTGACATGGGCTGTTTAATTTCCCTTTTCCAACCAAAAATACTGAAAACTTTTACATCGATATGGAAATCCGAGTTTGAAATTTGCGAT
6253861 6254066
0 2
Getting Gene
>Zerene_cesonia_Exon_10_NC_052102.1_6253861_6254066_left_0_right_2
GATTGGTCGGATCTAAtagaatatgttataaatacatttgtaaagAAACCACACACTTTGCAACGTACGGCAAGCTTCCGATTGGATACAGAAAGTTTCGATGACGACAAAATTCTTAAAGAACTTATTGAGAATGAGGAGAAACCAAATGAAGATGTGTCGTCTAGCAGAGCCATTGTTAACTATGATGCTAGCGTAAAGGACAG
6253193 6253668
1 1
Getting Gene
>Zerene_cesonia_Exon_11_NC_052102.1_6253193_6253668_left_1_right_1
atggACAGACGACGAACTATCGGCATTTAATTGGATTTACATGCAAAGCAGTACATCTACAGATACAGTTGGTGAAATTGTTCGACTGTTTAAGGAAGATGGAATAATCAAAACTCGCGATGCAGT

GTTGTGCGAGACGACGTCGGTCGCGTCAGGCCTGACGCGTATGTACGTCTCAGACGAGGAGTAG
>Leptidea_juvernica_Exon_16_CATIXQ010000011.1_11730490_11730553_left_1_right_0
GTTGTGCGAGACGACGTCGGTCGCGTCAGGCCTGACGCGTATGTACGTCTCAGACGAGGAGTAG
Leptidea_reali
GCA_949710795.1_LrealiM_genomic.fna
13842846 13842945
0 1
Getting Gene
MDGWILRGPLIHSTFSSLGFSDSKGYCINENCN
>Leptidea_reali_Exon_1_CATIXK010000015.1_13842846_13842945_left_0_right_1
ATGGATGGATGGATTCTTCGAGGTCCGCTAATTCACAGCACTTTCAGTAGCCTAGGATTTTCTGATTCAAAGGGATATTGCATAAATGAAAACTGTAATG
13841572 13842140
2 0
Getting Gene
>Leptidea_reali_Exon_2_CATIXK010000015.1_13841572_13842140_left_2_right_0
ATGCTTTGGAATgcatactacataatatactgaCAGAAGATAGGAATTTACGCACGTACAGAAGGGCTATAAGTTTCGGGCAAAATATCCAAAAAGATTTGATCCCCCTTTTGATAAATACAAAAGAAGACAAGACCGTAGAGCTCTTGATAAAGATTCTTGTCAATTTGACCATTCCTGTCGAGTGTTTACTCTCGATTAAGACGGCTTCGAACGCTGACGTTGGGAGACACACCATATTTGAGATTAACAGCCTATTGGCATCTACAAAGTCTGCATTCAAAGACGACAGAACAACAAAAGTTGTCATTGAATTTCTGAAAAAGAATGCTGAGCTAGATGTTGAGCTGAATGTTGAGCAGAAGTCAAAGCTATCACTCGAACAAT

7783994 7784094
0 2
Getting Gene
>Leptidea_sinapis_Exon_8_NC_066291.1_7783994_7784094_left_0_right_2
GACCTTGTCATCACAAATCATATCTTGTTGCTGTTATTGGACAATGTTATGAAGGTCTCGGACTACAAAGGCACCGGCACCATGGCTGACCATTTAATGCA
7787604 7787802
1 0
Getting Gene
>Leptidea_sinapis_Exon_9_NC_066291.1_7787604_7787802_left_1_right_0
GTTTGCAACACCTGAGATAATGTTCCAATACGGGCTACTGCTTGAAGACTATGAAAGTAATGGTGAGTTTGTGAATGACTGCATCTTCACCCTGATGCATCACGTGGGAGGTGATCTTGGCTGCTTGATCTCGCTCTTCCAACCCAACATATTGAAGACGTTCACCGCCATTTTTGAATCGGAATATAAAATTTGTGAG
7788255 7788448
0 2
Getting Gene
>Leptidea_sinapis_Exon_10_NC_066291.1_7788255_7788448_left_0_right_2
GAATGGTCTGATCTCATCCAATATTCAATTAATACATTTGTTAAGAAACCGCTAAGCTCGAAACGTGCAGCTAGCTTTCGAATGCTCACAGGATGtgttgatgatgatattattgTCAAAGATGTGTCCGAAGACGGCAAGGAAAAAGAAAATACATCTAAAAAGGACTTTAACGTAGAATCCGTCCCAGACGG
7789444 7789907
1 1
Getting Gene
>Leptidea_sinapis_Exon_11_NC_066291.1_7789444_7789907_left_1_right_1
ATGGACAAATCGAGAGCTTGAATCGTTCAACTGGAAGTACCTACAGAGCACTAACTCATCAGATATAGTCGGTGAGATATTGAGACTGCTTAAAGAA